In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


In [2]:
data = pd.read_csv(r'C:\Users\hp\Downloads\epilepsy_eeg-main\epilepsy_eeg-main\csv_data\Epileptic Seizure Recognition.csv')
data = data.drop(columns=['Unnamed'])  # Drop ID

# Make binary: 1 = seizure, 0 = non-seizure
data['y'] = data['y'].apply(lambda x: 1 if x == 1 else 0)

X = data.drop('y', axis=1).values
y = data['y'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X = X.reshape(-1, 1, 178)  # shape: (samples, channels, time)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)


In [4]:
class EEGSet(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

    def __len__(self):
        return len(self.X)

train_set = EEGSet(X_train, y_train)
test_set = EEGSet(X_test, y_test)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64)


In [5]:
class SimpleChronoNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Conv1d(1, 96, 3, padding=1)
        self.rnn1 = nn.GRU(96, 64, batch_first=True)
        self.rnn2 = nn.GRU(64, 32, batch_first=True)
        self.fc = nn.Linear(32, 2)
        self.drop = nn.Dropout(0.3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.cnn(x))          
        x = x.permute(0, 2, 1)              
        x, _ = self.rnn1(x)                 
        x, _ = self.rnn2(x)                 
        x = self.drop(x[:, -1, :])          
        return self.fc(x)                   


In [6]:
model = SimpleChronoNet()
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [7]:
for ep in range(15):
    model.train()
    for Xb, yb in train_loader:
        Xb, yb = Xb.to(device), yb.to(device)
        out = model(Xb)
        loss = loss_fn(out, yb)

        opt.zero_grad()
        loss.backward()
        opt.step()

    print(f"Epoch {ep+1}/15 | Loss: {loss.item():.4f}")


Epoch 1/15 | Loss: 0.0611
Epoch 2/15 | Loss: 0.0992
Epoch 3/15 | Loss: 0.1742
Epoch 4/15 | Loss: 0.0228
Epoch 5/15 | Loss: 0.0339
Epoch 6/15 | Loss: 0.2107
Epoch 7/15 | Loss: 0.1380
Epoch 8/15 | Loss: 0.0776
Epoch 9/15 | Loss: 0.1247
Epoch 10/15 | Loss: 0.0189
Epoch 11/15 | Loss: 0.1362
Epoch 12/15 | Loss: 0.0332
Epoch 13/15 | Loss: 0.0208
Epoch 14/15 | Loss: 0.0310
Epoch 15/15 | Loss: 0.0267


In [9]:
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for Xb, yb in test_loader:
        Xb = Xb.to(device)
        out = model(Xb)
        preds = torch.argmax(out, dim=1).cpu().numpy()
        y_pred += preds.tolist()
        y_true += yb.tolist()

acc = accuracy_score(y_true, y_pred)
print(f"Accuracy: {acc * 100:.2f}%")


Accuracy: 98.52%
